### C S 329E HW 4

## Decision Tree Classifier

## Kieran Cosgrove & Alec Biggerstaff

For this weeks homework we are going to explore ideas around decision tree implementation!  

We will implement some helper functions that would be necessary for a home-grown tree:
  - calc_entropy
  - calc_gini
  
and them test them out at given data splits. 
  
And finally, to perform predictive and descriptive analytics we use the [Decision Tree Classifier](https://scikit-learn.org/stable/modules/tree.html#classification) class.

  
For this assignment, the stopping condition will be the depth of the tree. The impurity measure can be either `Entropy` or `Gini`

To test our tree built from the Decision Tree Classifier class, we will revisit our Melbourne housing data (that has been cleaned and pruned) and use the files:

   - `melb_tree_train.csv` for training the decision tree
   - `melb_tree_test.csv` for testing the decision tree

There are 10 features in these dataframes that we can use to describe and predict the housing "Type", which is 'h' house, 'u' duplex, or 't' townhome. 

In [1]:
# import the libraries!
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from math import log2
from sklearn import tree # Yes, you get to use scikit-learn this assignment, but only when I say!
import graphviz

## Q1 Load the Data
Load in the melb_tree_train.csv into a dataframe, and split that dataframe into `df_X` containing the features of the data set (everything but `Type`), and `s_y`, the series containing just the label column (just `Type`)

In [2]:
df = pd.read_csv("melb_tree_train.csv")
df_X = df.drop("Type", axis=1)
s_y = df["Type"]

## Q2 Define the entropy function
Implement a function `calc_entropy` that takes the labels series, s_y, as a parameter. Implement using the definition on p128 in the DM book and only use pandas and log2 libraries

In [3]:
def calc_entropy(s_y):
    totals = s_y.value_counts()
    entropy = 0
    for total in totals:
        prob = total/totals.sum()
        entropy += -prob*log2(prob)
    return entropy

## Q3 Use the entropy function to
  - (a) Calculate the entropy of the entire training set
  - (b) Calculate the entropy of the three partitions formed from 
    * Landsize $\in$ [0,200]
    * Landsize $\in$ (200,450]
    * Landsize $\in$ (450, $\infty$)

In [4]:
calc_entropy(s_y)

1.5825333114261775

In [5]:
calc_entropy(df[df["Landsize"].any() and df["Landsize"] <= 200]["Type"])

1.3456432116206725

In [6]:
calc_entropy(df[(df["Landsize"] >= 200) & (df["Landsize"] <= 450)]["Type"])

1.4613003850147297

In [7]:
calc_entropy(df[(df["Landsize"] >= 450)]["Type"])

1.09954792005911

## Q4 Define the Gini Index
Implement the function `calc_gini` that takes the labels series, s_y, as a parameter. Implement using the definition on p128 in the DM book.  Use only pandas library

In [8]:
def calc_gini(s_y):
    totals = s_y.value_counts()
    gini = 0
    for total in totals:
        prob = total/totals.sum()
        gini += prob**2
    return 1 - gini

## Q5 Use the Gini Index function to
  - (a) Calculate the Gini index of the entire training set
  - (b) Calculate the Gini index of the three partitions formed from 
    * Landsize $\in$ [0,200]
    * Landsize $\in$ (200,450]
    * Landsize $\in$ (450, $\infty$)

In [9]:
calc_gini(s_y)

0.6655601280292638

In [10]:
calc_gini(df[df["Landsize"].any() and df["Landsize"] <= 200]["Type"])

0.5741211482422965

In [11]:
calc_gini(df[(df["Landsize"] >= 200) & (df["Landsize"] <= 450)]["Type"])

0.6077339178346639

In [12]:
calc_gini(df[(df["Landsize"] >= 450)]["Type"])

0.43819341715976323

## Q6 Create a decision tree 
Using [scikit-learn](https://scikit-learn.org/stable/modules/tree.html#tree) create a multi class classifer for the data set using the Entropy impurity measure and a max depth of 3

Note that scikit-learn's algorithm doesn't handle categorical data, so that needs to be preprocessed using an one hot encoding.

Display the tree using `export_text` from sklearn.tree, and use that information to write some descriptive analytics on the classification of houses.  For extra fun, use the export_graphviz to draw the graph (see documentation on the [scikit-learn webpage](https://scikit-learn.org/stable/modules/tree.html#classification)).  If you are on a Windows system and get stuck reach out in the homework channel to see if anyone else could get it working for tips and tricks.  If enough people with Windows can't make the visualization, I won't count it toward the homework grade.

In [13]:
type_cols = pd.get_dummies(df_X['CouncilArea'])
for col_title in type_cols:
    df_X[col_title] = type_cols[col_title]
del df_X['CouncilArea']

decision_tree = tree.DecisionTreeClassifier(criterion = 'entropy', max_depth = 3)
decision_tree = decision_tree.fit(df_X, s_y)
results = tree.export_text(decision_tree, feature_names = list(df_X.columns))
print(results)

|--- YearBuilt <= 1977.50
|   |--- Rooms <= 2.50
|   |   |--- Price <= 915000.00
|   |   |   |--- class: u
|   |   |--- Price >  915000.00
|   |   |   |--- class: h
|   |--- Rooms >  2.50
|   |   |--- Landsize <= 429.00
|   |   |   |--- class: h
|   |   |--- Landsize >  429.00
|   |   |   |--- class: h
|--- YearBuilt >  1977.50
|   |--- BuildingArea <= 104.31
|   |   |--- Landsize <= 76.00
|   |   |   |--- class: u
|   |   |--- Landsize >  76.00
|   |   |   |--- class: t
|   |--- BuildingArea >  104.31
|   |   |--- Landsize <= 391.00
|   |   |   |--- class: t
|   |   |--- Landsize >  391.00
|   |   |   |--- class: h



In [14]:
names = list(df_X)
dot_data = tree.export_graphviz(decision_tree, out_file=None, 
                                feature_names=names, 
                                filled=True, 
                                rounded=True, 
                                special_characters=True)  
graph = graphviz.Source(dot_data)  
graph 

ExecutableNotFound: failed to execute ['dot', '-Tsvg'], make sure the Graphviz executables are on your systems' PATH

Markdown explaination here

## Q7 Calculate the Accuracy and Display Learning Curve
Load in the test data.

Use the scikit-learn library to create decision trees with the following configurations

    - Vary the max depth from 2 to 15 with the Gini Index as the impurity measure
    - Vary the max depth from 2 to 15 with the Entropy as the impurity measure


Evaluate the accuracy of each decision tree with both the 
  - Training set
  - Test set

Display the results graphicaly, and offer an analysis of the trend (or if no trend present, offer a hypotheisis of why).  The max depth should be on the x-axis, and the error rate should be on the y-axis (see figure 3.23 in your DM textbook for a similar style of graph that uses leaf nodes instead of depth for the x-axis).